In [1]:
# 定義清除功能
import re

def word_filter(word):      
   #text.rstrip()
   word = word.replace("\n", "")
   word = word.replace("\r", "")
   word = re.sub(r'^\d+/\d+/\d+|\d+/\d+$', '', word)
   word = re.sub(r'^檔案樂活情報|回頁首|-|LOHAS|Archives\d+$', '', word)
   word = re.sub(r'^ALohas第\d+期|徵稿訊息|精采回顧|訂閱|取消訂閱|聯絡我們|│|\/$', '', word)
   word = re.sub(r'^D+alohas@archives.gov.tw$', '', word)  
   word = re.sub(r'^https?:\/\/alohas.archives.gov.tw\/\d+\/\D+.html$', '', word)    
   return word

def remove_subtitle(name):      
   name = name.replace(".html", "")
   name = name.replace(".pdf", "")
   name = name.replace(".docx", "")
   return name



In [ ]:
import os
import json
import pdfplumber


# Handle files in the directory
pdf_dir = "./docs/input/pdf/"
json_dir = "./docs/output/0_json/"
paths = os.listdir('./docs/input/pdf/')

for files in paths:
    all_out = []
    file_path = os.path.join(pdf_dir, files)

    if not os.path.exists(file_path):
        print(f"檔案不存在: {file_path}")
        continue  # 跳過不存在的檔案

    try:
        with pdfplumber.open(file_path) as pdf:
            paragraph = ""  # 用於合併不以 `。` 結尾的文字

            for i in range(len(pdf.pages)):
                data = pdf.pages[i]
                words = data.extract_words()
                print (words)  
                if words:
                    for unitword in words:
                        print (unitword)  # 印出每個單字的資訊
                        if unitword['x0'] < 400 and unitword['x1'] < 400:
                            text = word_filter(unitword['text'])  # 濾除不必要的內容                            
                            if text.endswith("。"):  # 如果句子以 `。` 結尾，視為完整段落
                                paragraph += text  # 加入當前句子
                                all_out.append(paragraph)  # 存入段落列表
                                paragraph = ""  # 重置段落
                            else:
                                paragraph += text  # 若非 `。` 結尾，暫存，等待合併
            # 確保最後的內容不會遺漏（例如最後一句沒有 `。`）
            if paragraph:
                all_out.append(paragraph)

        # ✅ 組裝 JSON 格式
        output_data = {
            "filename": files,
            "articles": all_out
        }

        output_filename = remove_subtitle(files) or "default_output"
        output_path = os.path.join(json_dir, output_filename + ".json")

        with open(output_path, "w", encoding="utf-8") as json_file:
            json.dump(output_data, json_file, ensure_ascii=False, indent=4)

        print(f"已成功處理並存儲：{output_path}")

    except Exception as exc:
        print(f"處理檔案 {files} 時發生錯誤: {exc}")


[{'text': '2018/6/5', 'x0': 26.453125, 'x1': 57.5937498, 'top': 15.695312479999984, 'doctop': 15.695312479999984, 'bottom': 23.695312479999984, 'upright': True, 'height': 8.0, 'width': 31.140624799999998, 'direction': 'ltr'}, {'text': '檔案樂活情報', 'x0': 266.17188, 'x1': 314.17188, 'top': 15.59375, 'doctop': 15.59375, 'bottom': 23.59375, 'upright': True, 'height': 8.0, 'width': 48.0, 'direction': 'ltr'}, {'text': 'LOHAS', 'x0': 316.39453624, 'x1': 343.51563, 'top': 15.695312479999984, 'doctop': 15.695312479999984, 'bottom': 23.695312479999984, 'upright': True, 'height': 8.0, 'width': 27.121093760000008, 'direction': 'ltr'}, {'text': '-', 'x0': 345.73828624, 'x1': 348.40234871999996, 'top': 15.695312479999984, 'doctop': 15.695312479999984, 'bottom': 23.695312479999984, 'upright': True, 'height': 8.0, 'width': 2.664062479999984, 'direction': 'ltr'}, {'text': 'Archives37', 'x0': 350.62500496, 'x1': 390.1992236, 'top': 15.695312479999984, 'doctop': 15.695312479999984, 'bottom': 23.695312479999